In [558]:
!pip install google-cloud-translate
import json
from google.cloud import translate_v2 as translate


# JSON 키 파일의 경로 설정
key_path = '/content/winged-fusion-394800-d055fe8afada.json'
# 번역 API 클라이언트 생성
client = translate.Client.from_service_account_json(key_path)



In [512]:
import pandas as pd

dataset_url = "/content/phishing_Data_18.tmx.json"

df_issues = pd.read_json(dataset_url, lines = True)

print(f"데이터프레임 크기: {df_issues.shape}")
df_issues = df_issues.T


import pandas as pd

# 데이터프레임 불러오기
dataset_url = "/content/phishing_Data_18.tmx.json"
df_issues = pd.read_json(dataset_url, lines=True)
print(f"데이터프레임 크기: {df_issues.shape}")

# Transpose
df_issues = df_issues.T

for i in range(len(df_issues)):
  if i == 0:
    df = pd.DataFrame({key: pd.Series(val) for key, val in df_issues[0][i].items()})
    continue
  df2 = pd.DataFrame({key: pd.Series(val) for key, val in df_issues[0][i].items()})
  df = pd.concat([df, df2], axis=0)
print(type(df))
# print(df)
# index 재설정
df.reset_index(drop=True, inplace = True)

데이터프레임 크기: (1, 131)
데이터프레임 크기: (1, 131)
<class 'pandas.core.frame.DataFrame'>


In [559]:
df

,text,index
0,ah yes,1.0
1,Yes (beep) Are you the customer?,NaN
2,yes,NaN
3,"Oh yes, I contacted you to inform you of the s...",NaN
4,yes,NaN
...,...,...
7417,yes,NaN
7418,In the case of the current payment certificate...,NaN
7419,yes,NaN
7420,"If not, this is because the legal team is curr...",NaN


In [517]:
# "text" 열에서 특정 키워드 삭제하는 함수
def remove_keywords(text):
    keywords = ["피해자:", "피해자 :", "사기범 :", "사기범:"]
    for keyword in keywords:
        text = text.replace(keyword, "")
    return text

# "text" 열의 값을 수정하여 기존 열을 대체
df['text'] = df['text'].apply(remove_keywords)

In [560]:
df

,text,index
0,ah yes,1.0
1,Yes (beep) Are you the customer?,NaN
2,yes,NaN
3,"Oh yes, I contacted you to inform you of the s...",NaN
4,yes,NaN
...,...,...
7417,yes,NaN
7418,In the case of the current payment certificate...,NaN
7419,yes,NaN
7420,"If not, this is because the legal team is curr...",NaN


In [519]:
# Translate API 초기화
client = translate.Client.from_service_account_json(key_path)
# 번역할 언어 설정
target_language = 'en'
# 각 행의 'text' 컬럼을 번역하여 새로운 컬럼에 저장
df['text'] = df['text'].apply(lambda text: client.translate(text, target_language=target_language)['translatedText'])

df

,text,index
0,ah yes,1.0
1,Yes (beep) Are you the customer?,NaN
2,yes,NaN
3,"Oh yes, I contacted you to inform you of the s...",NaN
4,yes,NaN
...,...,...
7417,yes,NaN
7418,In the case of the current payment certificate...,NaN
7419,yes,NaN
7420,"If not, this is because the legal team is curr...",NaN


In [557]:
df.drop('index',axis=1)

,text
0,ah yes
1,Yes (beep) Are you the customer?
2,yes
3,"Oh yes, I contacted you to inform you of the s..."
4,yes
...,...
7417,yes
7418,In the case of the current payment certificate...
7419,yes
7420,"If not, this is because the legal team is curr..."


In [430]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [431]:
!pip install --upgrade gensim --quiet

In [432]:
import gensim
print(gensim.__version__)
!pip install transformers --quiet

4.3.1


In [433]:
import transformers

In [434]:
!pip install sacremoses --quiet

In [435]:
import sacremoses

nlpaug 를 이용한 증강

In [436]:
!pip install numpy requests nlpaug
!pip install nltk>=3.4.5

In [437]:
import os
os.environ["MODEL_DIR"] = '../model'
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [438]:
from nlpaug.util.file.download import DownloadUtil



DownloadUtil.download_word2vec(dest_dir = '.')
DownloadUtil.download_fasttext(dest_dir = '.', model_name ='wiki-news-300d-1M')
DownloadUtil.download_glove(dest_dir = '.', model_name = 'glove.42B.300d')

Downloading...
From: https://drive.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
100%|██████████| 1.65G/1.65G [00:06<00:00, 238MB/s]


Substitute or insert word by contextual word embeddings

In [ ]:
## Substitute word by contextual word embeddings (BERT, DistilBERT, RoBERTA or XLNet)
aug = naw.ContextualWordEmbsAug(
  # Other models include 'distilbert-base-uncased', 'roberta-base', etc.
  model_path = 'bert-base-uncased',
  # You can also choose "insert"
  action = "substitute")

# 데이터 증강
augmented_texts = []
for text in df['text']:
    augmented_text = aug.augment(text)
    augmented_texts.append(augmented_text)

# 새로운 데이터프레임 생성 및 증강된 데이터 저장
augmented_data = {'text': augmented_texts}
augmented_df = pd.DataFrame(augmented_data)

# 기존 데이터프레임과 증강된 데이터프레임을 연결하여 증강된 데이터 추가
combined_df = pd.concat([df, augmented_df], ignore_index=True)


In [524]:
combined_df

,text
0,ah yes
1,Yes (beep) Are you the customer?
2,[then yes]
3,[yes ( 1 ) a you her customer?]


Substitute or insert word by synonym

In [525]:

## Substitute word by WordNet's synonym
aug = naw.SynonymAug(aug_src = 'wordnet')

# 새로운 데이터프레임 생성
augmented_texts1 = []

for text in df['text']:
    augmented_text = aug.augment(text)
    augmented_texts1.append(augmented_text)

# 새로운 데이터프레임 생성 및 증강된 데이터 저장
augmented_data1 = {'text': augmented_texts1}

augmented_df1 = pd.DataFrame(augmented_data1)

# 기존 데이터프레임과 증강된 데이터프레임을 연결하여 증강된 데이터 추가
combined_df1 = pd.concat([combined_df, augmented_df1], ignore_index=True)


In [526]:
combined_df1

,text
0,ah yes
1,Yes (beep) Are you the customer?
2,[then yes]
3,[yes ( 1 ) a you her customer?]
4,[ah yes]
5,[Yes (beep) Represent you the client?]


In [527]:
## Substitute word by contextual word embeddings (BERT, DistilBERT, RoBERTA or XLNet)
aug = naw.ContextualWordEmbsAug(
  # Other models include 'distilbert-base-uncased', 'roberta-base', etc.
  model_path = 'bert-base-uncased',
  # You can also choose "insert"
  action = "substitute")

# 데이터 증강
augmented_texts2= []
for text in df['text']:
    augmented_text = aug.augment(text)
    augmented_texts2.append(augmented_text)

# 새로운 데이터프레임 생성 및 증강된 데이터 저장
augmented_data2 = {'text': augmented_texts2}
augmented_df2 = pd.DataFrame(augmented_data2)

# 기존 데이터프레임과 증강된 데이터프레임을 연결하여 증강된 데이터 추가
combined_df2 = pd.concat([combined_df1, augmented_df2],ignore_index=True)


In [528]:

# 'text' 열의 리스트를 문자열로 변환
def list_to_string(item):
    if isinstance(item, list):
        return ', '.join(item)  # 리스트 요소를 문자열로 합침
    return item

combined_df2['text'] = combined_df2['text'].apply(list_to_string)

In [529]:
combined_df2

,text
0,ah yes
1,Yes (beep) Are you the customer?
2,then yes
3,yes ( 1 ) a you her customer?
4,ah yes
5,Yes (beep) Represent you the client?
6,ah bloody
7,hello ( beep ) are you summer assistant?


In [530]:

# Translate API 초기화
client = translate.Client.from_service_account_json(key_path)
# JSON 키 파일의 경로 설정
key_path = '/content/winged-fusion-394800-d055fe8afada.json'
# 번역 실행 및 결과 출력
target_language = 'ko'

translated_texts = []

# 번역 실행 함수
def translate_text(text, target_language='ko'):
   translation = client.translate(text, target_language=target_language)
   return translation['translatedText']

# 'Text' 열의 값을 번역하여 업데이트
combined_df2['text'] = combined_df2['text'].apply(translate_text)

combined_df2

,text
0,아 예
1,예(삐) 손님이세요?
2,그럼 예
3,예 ( 1 ) 당신은 그녀의 고객입니까?
4,아 예
5,예 (삐) 고객을 대표합니까?
6,아 빌어먹을
7,안녕하세요 ( 삐 ) 당신은 여름 비서입니까?


In [531]:
# '사기범:'와 '피해자:'를 추가하여 새로운 데이터프레임 생성
new_rows = []
for idx, row in combined_df2.iterrows():
    if idx % 2 == 0:  # 홀수 번째 행
        new_rows.append({'text': '사기범: ' + row['text']})
    else:  # 짝수 번째 행
        new_rows.append({'text': '피해자: ' + row['text']})

combined_df2 = pd.DataFrame(new_rows)

In [547]:
combined_df2

,text
0,사기범: 아 예
1,피해자: 예(삐) 손님이세요?
2,사기범: 그럼 예
3,피해자: 예 ( 1 ) 당신은 그녀의 고객입니까?
4,사기범: 아 예
5,피해자: 예 (삐) 고객을 대표합니까?
6,사기범: 아 빌어먹을
7,피해자: 안녕하세요 ( 삐 ) 당신은 여름 비서입니까?


In [548]:
new_rows = []
for i in range(0, len(combined_df2), 5):
    text_group = combined_df2['text'].iloc[i:i+5]
    combined_text = ', '.join(text_group)
    new_row = {'text': combined_text, 'index': 1}
    new_rows.append(new_row)

final_df = pd.DataFrame(new_rows)

In [549]:
final_df

,text,index
0,"사기범: 아 예, 피해자: 예(삐) 손님이세요?, 사기범: 그럼 예, 피해자: 예 ...",1
1,"피해자: 예 (삐) 고객을 대표합니까?, 사기범: 아 빌어먹을, 피해자: 안녕하세요...",1


In [555]:

# DataFrame을 JSON으로 변환
json_data = final_df.to_json(orient='records',lines=True,force_ascii=False)
json_data

'{"text":"사기범: 아 예, 피해자: 예(삐) 손님이세요?, 사기범: 그럼 예, 피해자: 예 ( 1 ) 당신은 그녀의 고객입니까?, 사기범: 아 예","index":1}\n{"text":"피해자: 예 (삐) 고객을 대표합니까?, 사기범: 아 빌어먹을, 피해자: 안녕하세요 ( 삐 ) 당신은 여름 비서입니까?","index":1}\n'